In [7]:
# .
import pytesseract
import requests, time
from selenium import webdriver
from PIL import Image
from bs4 import BeautifulSoup
import json

class DL_form:
            
   #-----------for headless chrome use this ------------#      
#         options = Options()
#         options.headless =True
#         self.browser = webdriver.Chrome("C:/Program Files/chromedriver.exe",chrome_options=options)
   #----------------------------------------------------#
    
    def fill(self):
        print("----------------------------------------")
        time.sleep(1)
        print("press 1 to Enter your image of driving license")
        print("                  or                        ")
        print("press 2 to Enter data Manually")
        time.sleep(1)
        try:
            press = int(input(">> "))
        except:
            print("try again,(input should be Integer)")
            press = int(input(">> "))
    #     print(press)
        if press == 1:
            print(":: you have choosen for image ::")
            print("@path example:-> D:\file\DL.img")
            DL_path = input("Give path of driving license image here  :-> ")
        else:
            print("")
            print(" :: you have chosen to enter data manually ::")
            print("")

            Dl = input("Enter Driving License No. : ")
            dob = input("Enter your date of birth here(eg.09-02-1997) :  ")
            time.sleep(1)
            print("")
            print(" :: collecting your given data...........! ::")
            print("")


        if press ==1:
            im = Image.open(r"driving license.jpg") 
        #     im = Image.open(DL_path)    

            # Cropped image 
            im1 = im.crop((110, 50, 225, 99)) 
            # im1.show() 
            im1.save("dlcrop.jpg")
            #for making it generalized,here we can use yolo(Deep Learning) for card detection and text detection

            img = Image.open('dlcrop.jpg')
            pytesseract.pytesseract.tesseract_cmd ='C:\Program Files\Tesseract-OCR/tesseract.exe'
            result = pytesseract.image_to_string(img) 
            with open('required_data.txt',mode ='w') as file:
                file.write(result)
        #         print(result)   
            file = open('required_data.txt')
            lines = file.readlines()
            DL = lines[0]
            DOB = lines[3].replace("/","-")
    #         print(DL)
    #         print(DOB)
        else :
            DL = Dl
            DOB = dob
    #         print(DL,DOB)

        
        browser = webdriver.Chrome("C:/Program Files/chromedriver.exe")
        browser.get('https://parivahan.gov.in/rcdlstatus/?pur_cd=101')
        time.sleep(3)
        
        elementID = browser.find_element_by_xpath("//input[@id='form_rcdl:tf_dlNO']")
        elementID.send_keys(DL)
        time.sleep(2)
        elementID = browser.find_element_by_xpath("//input[@id='form_rcdl:tf_dob_input']")
        elementID.send_keys(DOB)
        time.sleep(2)
        
        print("press 3 for entering captcha manually")
        print("               or                     ")
        print("press 4 for auto captcha filling using ocr-engine2(76% accuracy)")
        
        try:
            ocr = int(input())
        except:
            print(" :: input should be integer,enter again ::")
            ocr  = int(input())
        
        if ocr == 4 :
            #captcha img downloading
            element = browser.find_element_by_xpath("//img[@id='form_rcdl:j_idt34:j_idt41']")
            location = element.location
            size = element.size
            browser.save_screenshot("screenshot.png")
            x = location['x']
            y = location['y']
            width = location['x']+size['width']
            height = location['y']+size['height']
            im = Image.open('screenshot.png')
            im = im.crop((int(x), int(y), int(width), int(height)))
            im.save('captcha.png')
            
            #function callling 
            captcha = self.capt_read()
    #         print(captcha,"first")
            time.sleep(2)
            elementID = browser.find_element_by_xpath("//input[@id='form_rcdl:j_idt34:CaptchaID']")
            elementID.send_keys(captcha)
            elementID = browser.find_element_by_xpath("//span[contains(text(), 'Check Status')]").click()
            
            #check
            if browser.current_url == "https://parivahan.gov.in/rcdlstatus/?pur_cd=101":
                print("ocr failed to recognize the right captcha...!,try entering captcha manually")
                browser.close()
                return fill(self)
                time.sleep(2)
                print("restarting the process for ocr again......!")
                
        else :
            print(" :: you have chosen to enter captcha manually :: ")
            captcha = input("Enter the captcha as shown : ")
            try:
                elementID = browser.find_element_by_xpath("//input[@id='form_rcdl:j_idt34:CaptchaID']")
                time.sleep(1)
                elementID.send_keys(captcha)
                elementID = browser.find_element_by_xpath("//span[contains(text(), 'Check Status')]").click()
            except:
                if browser.current_url == "https://parivahan.gov.in/rcdlstatus/?pur_cd=101":
                    print("Entered Captcha is wrong, try again...")
                    captcha = input("Enter the captcha manually  : ")
                    elementID = browser.find_element_by_xpath("//input[@id='form_rcdl:j_idt34:CaptchaID']")
                    time.sleep(1)
                    elementID.send_keys(captcha)
                    elementID = browser.find_element_by_xpath("//span[contains(text(), 'Check Status')]").click()

            time.sleep(5)
            #read the result and save in json file
            src = browser.page_source
            soup = BeautifulSoup(src, 'lxml')
            table = soup.find('table', {'class': 'table table-responsive table-striped table-condensed table-bordered data-table'})
            table_body=table.find('tbody')
            rows = table_body.find_all('tr')
            for row in rows:
                cols=row.find_all('td')
                cols=[x.text.strip() for x in cols]
                break
            d_issue = cols[1][6:]   
            d_expiry = cols[2][3:]
            # print(d_issue,d_expiry)

            table2 =soup.find('div',{'class' : 'ui-datatable-tablewrapper'})
            table_body=table2.find('tbody')
            rows = table_body.find_all('tr')
            for row in rows:
                cols=row.find_all('td')
                cols=[x.text.strip() for x in cols]

            c_vehicle = cols[1]    
            # print(c_vehicle)


            dic = {
                "date of issue" : d_issue,
                "date of expiry" : d_expiry,
                "class of vehicle" : c_vehicle
            }

            file = open("result.json", "w")  

            json.dump(dic,file,indent = 6)  
            file.close()
            
            
                    
    def capt_read(self):
        chromeOptions = webdriver.ChromeOptions()
        prefs = {"download.default_directory" : "E:\Project\DL-form filling"}
        chromeOptions.add_experimental_option("prefs",prefs)
        browser = webdriver.Chrome(executable_path="C:/Program Files/chromedriver.exe", chrome_options=chromeOptions)    
        
        # using online ocr
        browser.get('https://ocr.space/')
        fileinput = browser.find_element_by_xpath("//input[@id='imageFile']")
        fileinput.send_keys('E:\Project\DL-form filling\captcha.png')
        #filtering for text and engine 2
        txt_button = browser.find_element_by_xpath("//input[@id='NoSearchable']").click()
        eng2_button = browser.find_element_by_xpath("//input[@id='engine2']").click()
        #startocr
        time.sleep(1)
        browser.execute_script("window.scrollTo(0, 1000)") 
        start_ocr = browser.find_element_by_xpath("//a[@onclick='StartOCR()']").click()   
        time.sleep(4)
        # download converted_txt
        try:
            os.remove(r'parsedResult.txt') #delete previous file
            print("previous text file deleted")
        except:
            pass

        download = browser.find_element_by_xpath("//a[@id='btnDownloadText']").click()
        # read_downloaded_txt_file
        time.sleep(3)
        file = open('parsedResult.txt')
        lines = file.readlines()
        try:
            captchar = lines[2]
            print(captchar,"second")
            return captchar
        except:
            pass
        time.sleep(2)
        browser.close()

        
        
        
 
obj = DL_form()
obj.fill()
# obj.capt_read()

----------------------------------------
press 1 to Enter your image of driving license
                  or                        
press 2 to Enter data Manually
>> 1
:: you have choosen for image ::
@path example:-> D:ile\DL.img
Give path of driving license image here  :-> eifbuewf
press 3 for entering captcha manually
               or                     
press 4 for auto captcha filling using ocr-engine2(76% accuracy)
3
 :: you have chosen to enter captcha manually :: 
Enter the captcha as shown : fg8km


In [43]:
import os
os.remove(r'parsedResult.txt') #delete previous file